In [1]:
import numpy as np
from tqdm import tqdm
import xarray as xr
import matplotlib.pyplot as plt
import warnings
from itertools import product
from polarization_controller import optical_fiber, polarization_splitter_rotator, mzi, phase_aligner, polarization_controller
from scipy.optimize import minimize
warnings.filterwarnings("ignore")
plt.style.use("plot_style.mplstyle")

# 

In [2]:
p_bidi = polarization_controller(tx_num_mzi_stages = 1)

In [3]:
sample_points = 201

phase_shifts = np.linspace(-np.pi, np.pi, sample_points)

rotation_rx = phase_shifts[np.random.permutation(sample_points)]
rotation_ry = phase_shifts[np.random.permutation(sample_points)]
rotation_rz = phase_shifts[np.random.permutation(sample_points)]
rotation_delta = phase_shifts[np.random.permutation(sample_points)]

p_bidi.fiber_1._rotation = (rotation_rx[0], rotation_ry[0], rotation_rz[0], rotation_delta[0])
p_bidi.fiber_2._rotation = (rotation_rx[1], rotation_ry[1], rotation_rz[1], rotation_delta[1])
p_bidi.fiber_3._rotation = (rotation_rx[2], rotation_ry[2], rotation_rz[2], rotation_delta[2])
p_bidi.fiber_4._rotation = (rotation_rx[3], rotation_ry[3], rotation_rz[3], rotation_delta[3])
p_bidi.fiber_5._rotation = (rotation_rx[4], rotation_ry[4], rotation_rz[4], rotation_delta[4])
p_bidi.fiber_6._rotation = (rotation_rx[5], rotation_ry[5], rotation_rz[5], rotation_delta[5])
p_bidi.fiber_7._rotation = (rotation_rx[6], rotation_ry[6], rotation_rz[6], rotation_delta[6])

p_bidi.oc_1._return_loss = -35
p_bidi.oc_2._return_loss = -35
p_bidi.oc_3._return_loss = -35
p_bidi.oc_4._return_loss = -35
p_bidi.oc_5._return_loss = -35
p_bidi.oc_6._return_loss = -35

p_bidi.recursive_update()

In [4]:
no_sweep = 61
voltages = np.linspace(-3.0, 3.0, no_sweep)

transmit_tm_forward = []
transmit_te_forward  = []
reflect_te_forward = []
reflect_tm_forward = []

transmit_tm_reverse = []
transmit_te_reverse  = []
reflect_te_reverse = []
reflect_tm_reverse = []

input_state_forward = np.array([1, 0, 0, 0])
input_state_reverse = np.array([0, 0, 1, 0])

for tx_pa_voltage, tx_mzi_voltage in tqdm(product(voltages, voltages), total=no_sweep**2):
  p_bidi.bidi_tx.pa.XPS1.heater_voltage = 0.0
  p_bidi.bidi_tx.pa.XPS2.heater_voltage = 0.0
  if tx_pa_voltage > 0.0:
    p_bidi.bidi_tx.pa.XPS1.heater_voltage = tx_pa_voltage
  if tx_pa_voltage < 0.0:
    p_bidi.bidi_tx.pa.XPS2.heater_voltage = -tx_pa_voltage


  p_bidi.bidi_tx.mzi_1.XPS1.heater_voltage = 0.0
  p_bidi.bidi_tx.mzi_1.XPS2.heater_voltage = 0.0
  if tx_mzi_voltage > 0.0:
    p_bidi.bidi_tx.mzi_1.XPS1.heater_voltage = tx_mzi_voltage
  if tx_mzi_voltage < 0.0:
    p_bidi.bidi_tx.mzi_1.XPS2.heater_voltage = -tx_mzi_voltage
    
  p_bidi.recursive_update()

  output_state_forward = p_bidi.smatrix @ input_state_forward
  
  transmit_te_forward.append(np.abs(output_state_forward[2])**2)
  transmit_tm_forward.append(np.abs(output_state_forward[3])**2)
  reflect_te_forward.append(np.abs(output_state_forward[0])**2)
  reflect_tm_forward.append(np.abs(output_state_forward[1])**2)

  output_state_reverse = p_bidi.smatrix @ input_state_reverse
  
  transmit_te_reverse.append(np.abs(output_state_reverse[0])**2)
  transmit_tm_reverse.append(np.abs(output_state_reverse[1])**2)
  reflect_te_reverse.append(np.abs(output_state_reverse[2])**2)
  reflect_tm_reverse.append(np.abs(output_state_reverse[3])**2)

100%|██████████| 3721/3721 [00:02<00:00, 1661.81it/s]


In [5]:
index_mzi = np.argmax(transmit_tm_forward)%no_sweep
index_pa = int(np.argmax(transmit_tm_forward)/no_sweep)
print(voltages[index_mzi], voltages[index_pa])

-1.9 -2.4


In [6]:
p_bidi.bidi_tx.pa.XPS1.heater_voltage = 0.0
p_bidi.bidi_rx.pa.XPS2.heater_voltage = 0.0

if voltages[index_pa] > 0.0:
  p_bidi.bidi_tx.pa.XPS1.heater_voltage = voltages[index_pa]
if voltages[index_pa] < 0.0:
  p_bidi.bidi_tx.pa.XPS2.heater_voltage = -voltages[index_pa]

p_bidi.bidi_tx.mzi_1.XPS1.heater_voltage = 0.0
p_bidi.bidi_tx.mzi_1.XPS2.heater_voltage = 0.0

if voltages[index_mzi] > 0.0:
  p_bidi.bidi_tx.mzi_1.XPS1.heater_voltage = voltages[index_mzi]
if voltages[index_mzi] < 0.0:
  p_bidi.bidi_tx.mzi_1.XPS2.heater_voltage = -voltages[index_mzi]

p_bidi.recursive_update()

In [7]:
input_state_forward = np.array([1, 0, 0, 0])
output_state_forward = p_bidi.smatrix @ input_state_forward
print("Output state direction: ", np.abs(output_state_forward)**2)
print("Output state direction in log: ", 10*np.log10(np.abs(output_state_forward)**2))

input_state_reverse = np.array([0, 0, 1, 0])
output_state_reverse = p_bidi.smatrix @ input_state_reverse
print("Output state reverse direction: ", np.abs(output_state_reverse)**2)
print("Output state direction in log: ", 10*np.log10(np.abs(output_state_reverse)**2))

Output state direction:  [5.39756367e-05 1.16367698e-05 1.69432865e-04 9.44560402e-02]
Output state direction in log:  [-42.67802226 -49.34167556 -37.71002345 -10.24770265]
Output state reverse direction:  [1.69432865e-04 1.03385144e-01 6.94911312e-05 4.41707693e-05]
Output state direction in log:  [-37.71002345  -9.85541864 -41.58070619 -43.54865037]
